# <span style="color:Blue"><center> **Esquema BD - MariaDB**</center></span>

* María Camila Góngora Albán
******

## <span style="color:Blue"> Introducción</span>

_Statistics Finland_ fue fundada en 1865 y es la única autoridad pública finlandesa establecida específicamente para las estadísticas. Produce la gran mayoría de las estadísticas oficiales finlandesas y es un actor internacional importante en el campo de las estadísticas.

_Statistics Finland_ combina los datos recopilados con su propia experiencia para producir estadísticas y servicios de información. Por ejemplo recopila estadísticas internacionales tomando como fuente los datos y fuentes de datos producidos por organismos y organizaciones internacionales más importantes y fiables.

<img src="SFIN.PNG">(Tomada de [Statistics Finland](http://tilastokeskus.fi/org/index_en.html))

Dentro del conjunto de datos disponibles, se encuentra la base de datos _world_ la cual contiene el listado de paises junto con algunas de sus características, como por ejemplo ubicación, idioma y densidad poblacional. En este documento se empleará una muestra de esta base, la cual se encuentra disponible en [world database](https://dev.mysql.com/doc/index-other.html).

## <span style="color:Blue"> Instalación</span>

La base de datos de muestra de _world_ se encuentra en un archivo descargable de la fuente indicada y está disponible en un archivo tar comprimido o en formato Zip, el cual contiene un archivo:  world.sql

El archivo world.sql contiene todas las declaraciones CREATE necesarias para crear la estructura de la base de datos, incluidas las siguientes tablas, con la descripción de los campos e identificación de llaves primarias:

* country: información de países del mundo
* city: información de algunas ciudades en los países
* country_language: idiomas hablados en cada país

In [2]:
mysql -u root -p

Your SQL code doesn't end with delimiter `;`


In [3]:
SOURCE /home/mcgongoraa/Documentos/BigData/Tarea Esquema BD/world.sql;

Query OK

In [4]:
USE world;

Query OK

In [5]:
SHOW FULL TABLES;

Tables_in_world,Table_type
city,BASE TABLE
country,BASE TABLE
countrylanguage,BASE TABLE


## <span style="color:Blue"> Descripción general</span>

La tabla _country_ contiene 239 países, _ciudad_ contiene 4079 ciudades asociadas a los países listados en _country_

In [6]:
SELECT COUNT(*) FROM country;
SELECT COUNT(*) FROM countrylanguage;
SELECT COUNT(*) FROM city;

COUNT(*)
239
COUNT(*)
984
COUNT(*)
4079


La tabla _city_ tiene 5 atributos, todos obligatorios:
* ID (Llave primaria, la cual se crea de forma automática incrementando)
* Name
* CountryCode (Llave foránea)
* District
* Population

In [7]:
SELECT * FROM city limit 5;

ID,Name,CountryCode,District,Population
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500
3,Herat,AFG,Herat,186800
4,Mazar-e-Sharif,AFG,Balkh,127800
5,Amsterdam,NLD,Noord-Holland,731200


In [8]:
DESCRIBE city;

Field,Type,Null,Key,Default,Extra
ID,int(11),NO,PRI,NULL,auto_increment
Name,char(35),NO,,,
CountryCode,char(3),NO,MUL,,
District,char(20),NO,,,
Population,int(11),NO,,0,


La tabla _country_ tiene 15 atributos, dentro de los cuales se encuentra la llave primaria es:
* Code

Las únicos atributos opcionales son: IndepYear, LifeExpectancy, GNP, GNPOld, HeadOfState y Capital.

El atributo Continent tiene unos valores predefinidos como opcionales.

In [8]:
SELECT * FROM country limit 5;

Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
ABW,Aruba,North America,Caribbean,193.00,NULL,103000,78.4,828.00,793.00,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129,AW
AFG,Afghanistan,Asia,Southern and Central Asia,652090.00,1919,22720000,45.9,5976.00,NULL,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1,AF
AGO,Angola,Africa,Central Africa,1246700.00,1975,12878000,38.3,6648.00,7984.00,Angola,Republic,José Eduardo dos Santos,56,AO
AIA,Anguilla,North America,Caribbean,96.00,NULL,8000,76.1,63.20,NULL,Anguilla,Dependent Territory of the UK,Elisabeth II,62,AI
ALB,Albania,Europe,Southern Europe,28748.00,1912,3401200,71.6,3205.00,2500.00,Shqipëria,Republic,Rexhep Mejdani,34,AL


In [9]:
DESCRIBE country;

Field,Type,Null,Key,Default,Extra
Code,char(3),NO,PRI,,
Name,char(52),NO,,,
Continent,"enum('Asia','Europe','North America','Africa','Oceania','Antarctica','South America')",NO,,Asia,
Region,char(26),NO,,,
SurfaceArea,"decimal(10,2)",NO,,0.00,
IndepYear,smallint(6),YES,,NULL,
Population,int(11),NO,,0,
LifeExpectancy,"decimal(3,1)",YES,,NULL,
GNP,"decimal(10,2)",YES,,NULL,
GNPOld,"decimal(10,2)",YES,,NULL,


La tabla _countrylanguage_ contiene 4 atributos, todos obligatorios:
* CountryCode (Llave primaria)
* Language (Llave primaria)
* IsOfficial
* Percentage

In [9]:
SELECT * FROM countrylanguage limit 5;

CountryCode,Language,IsOfficial,Percentage
ABW,Dutch,T,5.3
ABW,English,F,9.5
ABW,Papiamento,F,76.7
ABW,Spanish,F,7.4
AFG,Balochi,F,0.9


In [10]:
DESCRIBE countrylanguage;

Field,Type,Null,Key,Default,Extra
CountryCode,char(3),NO,PRI,,
Language,char(30),NO,PRI,,
IsOfficial,"enum('T','F')",NO,,F,
Percentage,"decimal(4,1)",NO,,0.0,


## <span style="color:Blue"> Diagrama Entidad Relación </span>

Se identifican principalmente tres entidades: País, Ciudad e Idioma. Donde las relaciones entre ellas son:

* País - Ciudad: Relación opcional uno a muchos 
* País - Idioma: Relación obligatoria uno a muchos

En el diagrama a continuación se evidencian estas relaciones. Adicionalmente se pueden ver los atributos correspondientes a cada entidad y se pueden identificar las llaves primarias.

<img src="ER_world.png">(Construida en DBeaver)

## <span style="color:Blue"> Procedimiento almacenado, Función y Trigger </span>

### <span style="color:Green"> Procedimiento Almacenado </span>

Un procedimiento almacenado es un conjunto de instrucciones de SQL en un único plan de ejecución, son llamados también, "store procedures". Se encuentran almacenados en la base de datos y pueden ser ejecutados en cualquier momento. [Def procedimiento](https://www.usmp.edu.pe/publicaciones/boletin/fia/info41/procedimiento.html )

El siguiente procedimiento devuelve todos los idiomas por país y continente

In [38]:
DROP PROCEDURE IF EXISTS pais_continente_idioma;

CREATE procedure pais_continente_idioma()
BEGIN
select y.Continent,y.Name,x.*
from countrylanguage x
inner join country y
on x.countrycode=y.code;
END;

Query OK

In [91]:
call pais_continente_idioma();

Continent,Name,CountryCode,Language,IsOfficial,Percentage
North America,Aruba,ABW,Dutch,T,5.3
North America,Aruba,ABW,English,F,9.5
North America,Aruba,ABW,Papiamento,F,76.7
North America,Aruba,ABW,Spanish,F,7.4
Asia,Afghanistan,AFG,Balochi,F,0.9
Asia,Afghanistan,AFG,Dari,T,32.1
Asia,Afghanistan,AFG,Pashto,T,52.4
Asia,Afghanistan,AFG,Turkmenian,F,1.9
Asia,Afghanistan,AFG,Uzbek,F,8.8
Africa,Angola,AGO,Ambo,F,2.4


El siguiente procedimiento devuelve todos los idiomas por país y continente, filtrando por el Continente de interés

In [35]:
DROP PROCEDURE IF EXISTS continente_idioma;

CREATE procedure continente_idioma(cont_cod varchar(13))
BEGIN
select y.Continent,y.Name,x.*
from countrylanguage x
inner join country y
on x.countrycode=y.code
where Continent = cont_cod;
END;


Query OK

In [37]:
call continente_idioma('South America');

Continent,Name,CountryCode,Language,IsOfficial,Percentage
South America,Argentina,ARG,Indian Languages,F,0.3
South America,Argentina,ARG,Italian,F,1.7
South America,Argentina,ARG,Spanish,T,96.8
South America,Bolivia,BOL,Aimará,T,3.2
South America,Bolivia,BOL,Guaraní,F,0.1
South America,Bolivia,BOL,Ketšua,T,8.1
South America,Bolivia,BOL,Spanish,T,87.7
South America,Brazil,BRA,German,F,0.5
South America,Brazil,BRA,Indian Languages,F,0.2
South America,Brazil,BRA,Italian,F,0.4


### <span style="color:Green"> Función </span>

Una función es un conjunto de instrucciones SQL que realizan una tarea específica de manera automática al invocarla. Las funciones fomentan la reutilización del código, acepta entradas en forma de parámetros y devuelve un valor o tabla, según se defina.

La siguiente función suma el área de la superficie para el continente de interés.

In [55]:
DROP FUNCTION IF EXISTS area_continente;

CREATE FUNCTION area_continente(cont_cod varchar(13))
RETURNS float
RETURN (SELECT sum(SurfaceArea)
        FROM country
        WHERE Continent = cont_cod
        GROUP BY Continent);

Query OK

In [56]:
SELECT area_continente('Africa');

area_continente('Africa')
30250400


In [57]:
SELECT area_continente('Antarctica');

area_continente('Antarctica')
13132100


In [59]:
SELECT area_continente('Europe');

area_continente('Europe')
23049100


### <span style="color:Green"> Trigger </span>

Un Trigger o disparador, es un tipo especial de procedimiento almacenado que se ejecuta automáticamente cuando ocurre un evento en el servidor de la base de datos. 

Según el tipo de evento que los desencadena se clasifican en:
*  Desencadenadores DML: se ejecutan cuando un usuario intenta modificar datos mediante un evento de lenguaje de manipulación de datos (DML). Por ejemplo, a través de INSERT, UPDATE o DELETE de una tabla o vista
*  Desencadenadores DDL: se ejecutan en respuesta a una variedad de eventos de lenguaje de definición de datos (DDL). Principalmente corresponde a las instrucciones CREATE, ALTER y DROP de SQL, y a determinados procedimientos almacenados del sistema que ejecutan operaciones de tipo DDL.
*  Desencadenadores LOGON: se activan en respuesta al evento LOGON que se genera cuando se establece la sesión de un usuario.

Como ejemplo se creará un trigger el cual, antes de modifcar la columna de población, lo registre en un log ([MariaDB ejemplo](https://www.mariadbtutorial.com/mariadb-triggers/mariadb-create-trigger/)). 
Para tal fin, se crea una copia de la tabla _country_

In [63]:
create table country_copy
select * 
from country;

Query OK

Se crea la tabla _Population_logs_ para registrar los cambios en la columna de población de la tabla _country_

In [78]:
drop table if exists population_logs;

create table population_logs(
    log_id int auto_increment,
    Code char(3) not null,
    old_population int not null,
    new_population int not null,
    updated_at timestamp default current_timestamp,
    primary key(log_id)
);

Query OK

Se crea un Trigger invocado antes de que se realice un cambio en la tabla _country_.

In [80]:
drop trigger if exists before_country_update;

create trigger before_country_update 
    before update on country_copy
    for each row
    insert into population_logs(
        Code, 
        old_population, 
        new_population
    )
    values(
        old.Code,
        old.population,
        new.population
    );

Query OK

El valor original para la población de Colombia es

In [67]:
select Code,Name,Population 
from country
where Code = "COL";

Code,Name,Population
COL,Colombia,42321000


Actualizar la población para Colombia sobre la tabla que se creó como copia

In [81]:
update 
    country_copy
set 
    population = 1352617399
where 
    Code = 'COL';

Query OK

La actualización realizada sobre la tabla _country_copy_ se realiza después de que el Trigger creado se activa, insertando una nueva línea en la tabla _population_logs_

In [82]:
select * 
from population_logs;

log_id,Code,old_population,new_population,updated_at
1,COL,42321000,1352617399,2021-04-12 22:58:10


## <span style="color:Blue"> Consultas </span>

In [95]:
create table Continent_language 
as select y.Continent,y.Name,x.*
from countrylanguage x
inner join country y
on x.countrycode=y.code;

Query OK

In [96]:
SELECT * 
FROM Continent_language 
WHERE language="Spanish";

Continent,Name,CountryCode,Language,IsOfficial,Percentage
North America,Aruba,ABW,Spanish,F,7.4
Europe,Andorra,AND,Spanish,F,44.6
South America,Argentina,ARG,Spanish,T,96.8
North America,Belize,BLZ,Spanish,F,31.6
South America,Bolivia,BOL,Spanish,T,87.7
North America,Canada,CAN,Spanish,F,0.7
South America,Chile,CHL,Spanish,T,89.7
South America,Colombia,COL,Spanish,T,99.0
North America,Costa Rica,CRI,Spanish,T,97.5
North America,Cuba,CUB,Spanish,T,100.0


In [98]:
create table Country_city 
as select x.*,y.id as id_city,y.Name as Name_city,y.District,y.Population as Population_city
from country x
inner join city y
on x.code =y.countrycode;

Query OK

In [101]:
select Name_city,District,Population_city
from 
Country_city
where code='SWE'
order by Population_city;

Name_city,District,Population_city
Gävle,Gävleborgs län,90742
Sundsvall,Västernorrlands län,93126
Borås,West Götanmaan län,96883
Lund,Skåne län,98948
Umeå,Västerbottens län,104512
Jönköping,Jönköpings län,117095
Helsingborg,Skåne län,117737
Norrköping,East Götanmaan län,122199
Örebro,Örebros län,124207
Västerås,Västmanlands län,126328


In [ ]:
/*%%magic_python
from matplotlib import pyplot
x = select * from country;
print(x)

## <span style="color:Blue"> Bibliografía</span>

* http://tilastokeskus.fi/org/index_en.html
* https://dev.mysql.com/doc/world-setup/en/
* https://dev.mysql.com/doc/index-other.html

* https://www.usmp.edu.pe/publicaciones/boletin/fia/info41/procedimiento.html (procedimiento almacenado)
* https://www.aulaclic.es/sqlserver/t_9_8.htm (trigger)